In [ ]:
# !pip install gradio

In [1]:
import gradio as gr


In [2]:
def combined_pipeline(image):
    try:
        # Step 1: Crop the detected object using Roboflow
        cropped_image = crop_detected_object(image)

        if isinstance(cropped_image, str):  # Check if the result is an error message
            return cropped_image

        # Step 2: Run Florence-2 inference on the cropped image
        extracted_text = run_inference(cropped_image)

        # Step 3: Extract book details from the text using ChatGPT
        details = extract_details(extracted_text)

        return details
    except Exception as e:
        return f"Error in combined_pipeline: {str(e)}"

In [3]:
interface = gr.Interface(
    fn=combined_pipeline,
    inputs=gr.Image(type="pil", mirror_webcam=False),
    outputs=gr.Textbox(),
    title="Object Detection and Image Inference with Book Details Extraction",
    description="Upload an image to detect objects using Roboflow, crop the detected object, run Florence-2 inference, and extract book details using GPT-4o."
)

In [4]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860


KeyboardInterrupt: 

In [5]:
interface.close()

Closing server running on port: 7860


In [6]:
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /home/ibnu/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/ibnu/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:58<00:00, 796kB/s] 


In [7]:
import requests
from PIL import Image
from torchvision import transforms

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def predict(inp):
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
    confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
  return confidences

In [9]:
import gradio as gr

interface = gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=3),
             examples=["lion.jpg", "cheetah.jpg"]).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [10]:
interface.close()

AttributeError: 'TupleNoPrint' object has no attribute 'close'